In [23]:
# Завнтажуємо дані
import pandas as pd

#Завантаження даних
file_path = 'internet_service_churn.csv'
data = pd.read_csv(file_path)

#Первинний огляд даних
print(data.head(5))
print(data.shape)

   id  is_tv_subscriber  is_movie_package_subscriber  subscription_age  \
0  15                 1                            0             11.95   
1  18                 0                            0              8.22   
2  23                 1                            0              8.91   
3  27                 0                            0              6.87   
4  34                 0                            0              6.39   

   bill_avg  reamining_contract  service_failure_count  download_avg  \
0        25                0.14                      0           8.4   
1         0                 NaN                      0           0.0   
2        16                0.00                      0          13.7   
3        21                 NaN                      1           0.0   
4         0                 NaN                      0           0.0   

   upload_avg  download_over_limit  churn  
0         2.3                    0      0  
1         0.0                    0

***Обробка пропусків***

In [26]:
#Аналіз відсутніх значень у наборі даних
missing_data = data.isnull().sum()

#Обчислення відсотка відсутніх значень для кожного стовпчика
missing_percentage = (missing_data / len(data)) * 100

#Відображення стовпців з відсутніми значеннями та їх відсотки
missing_info = pd.DataFrame({"Missing Values": missing_data, "Percentage": missing_percentage})
missing_info[missing_info["Missing Values"] > 0]

,Missing Values,Percentage
reamining_contract,21572,29.847525
download_avg,381,0.527161
upload_avg,381,0.527161


**Було обрано заповнення медіанами, тому що цей підхід дозволяє зберегти дані без значного внеску в статистику. Якщо б пропущені значення видалили, це б вплинуло на логіку аналізу, оскільки дані могли бути критичними для розуміння клієнтів.**

In [29]:
# Список колонок для заповнення пропусків медіанами
columns_to_fill = ['download_avg', 'upload_avg']

# Заповнюємо пропуски медіанами
for column in columns_to_fill:
    data[column] = data[column].fillna(data[column].median())

**Виходячи з аналізу даних, для користувачів з відтоком (churn = 1) значення remaining_contract мають схильність до низьких значень, тоді як для користувачів без відтоку (churn = 0) значення більше розподілені й ближчі до середніх значень, тому ми будемо заповнювати їх за допомогою групового розподілу, використовуючи окремо медіани для churn = 1 та для churn = 0**

In [32]:
# Окрема обробка для 'reamining_contract', із-за великої кількості пропущених даних і виявлення впливу на цільову змінну

target = 'reamining_contract'  # Змінна з пропусками
grouping_var = 'churn'  # Змінна для групування

# Функція для заповнення пропусків медіаною у кожній групі
def fill_missing_by_group(data, target, grouping_var):
    for group in data[grouping_var].unique():
        group_median = data.loc[data[grouping_var] == group, target].median()
        data.loc[(data[grouping_var] == group) & (data[target].isnull()), target] = group_median
    return data

# Виклик функції для заповнення пропусків
data = fill_missing_by_group(data, target, grouping_var)

print("Пропущені значення заповнені медіаною окремо для кожної групи.")

Пропущені значення заповнені медіаною окремо для кожної групи.


In [34]:
# Перевірка, що відсутніх значень більше немає
missing_values_count = data.isnull().sum()
print(missing_values_count)

id                             0
is_tv_subscriber               0
is_movie_package_subscriber    0
subscription_age               0
bill_avg                       0
reamining_contract             0
service_failure_count          0
download_avg                   0
upload_avg                     0
download_over_limit            0
churn                          0
dtype: int64


***Закодувати категоріальні змінні***

**У датасеті всі потенційно категоріальні змінні is_tv_subscriber, is_movie_package_subscriber, download_over_limit, churn вже представлені як числові бінарні змінні 0 або 1.
Це означає, що вони не потребують додаткового кодування.**

***Провести нормалізацію/стандартизацію числових ознак***

**У нашому випадку стандартизація буде більш доречною для забезпечення збалансованості вкладу різних ознак.**

In [40]:
from sklearn.preprocessing import StandardScaler

#Визначення числових змінних
numerical_columns = ['subscription_age', 'bill_avg', 'reamining_contract', 'download_avg', 'upload_avg']

#Ініціалізація StandardScaler
scaler = StandardScaler()

#Стандартизація
data[numerical_columns] = scaler.fit_transform(data[numerical_columns])

#Вивід результату
print(data.head())

   id  is_tv_subscriber  is_movie_package_subscriber  subscription_age  \
0  15                 1                            0          4.668335   
1  18                 0                            0          2.835389   
2  23                 1                            0          3.174460   
3  27                 0                            0          2.171991   
4  34                 0                            0          1.936116   

   bill_avg  reamining_contract  service_failure_count  download_avg  \
0  0.458372           -0.581665                      0     -0.556631   
1 -1.433376           -0.790463                      0     -0.689440   
2 -0.222657           -0.790463                      0     -0.472835   
3  0.155692           -0.790463                      1     -0.689440   
4 -1.433376           -0.790463                      0     -0.689440   

   upload_avg  download_over_limit  churn  
0   -0.192060                    0      0  
1   -0.426895                    0

In [42]:
#Визначення числових змінних
numerical_columns = ['subscription_age', 'bill_avg', 'reamining_contract', 'download_avg', 'upload_avg']
# Перевірка правильності
print(data[numerical_columns].mean()) #Середнє значення
print(data[numerical_columns].std()) #Стандартне відхилення

subscription_age      0.000000e+00
bill_avg              7.864989e-18
reamining_contract   -1.006719e-16
download_avg          1.101098e-17
upload_avg            3.145995e-17
dtype: float64
subscription_age      1.000007
bill_avg              1.000007
reamining_contract    1.000007
download_avg          1.000007
upload_avg            1.000007
dtype: float64


In [44]:
# Збереження параметрів StandardScaler для подальшого використання
scaler_path = 'scaler.pkl'
joblib.dump(scaler, scaler_path)
print(f"Параметри StandardScaler збережено у файл: {scaler_path}")

Параметри StandardScaler збережено у файл: scaler.pkl


In [21]:
#Збереження підготовлених данних
data.to_csv('processed_data.csv', index=False)